In [ ]:
!pip install ir-datasets scikit-learn

# Laboratorio #3: Medidas de evaluación

Las medidas de evaluación desempeñan un papel fundamental en los Sistemas de Recuperación de Información (SRI) al proporcionar una forma objetiva de medir su rendimiento y efectividad. Estas medidas permiten a las personas comprender y mejorar la calidad de los resultados recuperados por el sistema.

Como importancia del uso de las medidas de evaluación, se pueden mencionar:

1. **Calidad de los Resultados**: Las medidas de evaluación permiten determinar la precisión y relevancia de los documentos recuperados en relación con las consultas de los usuarios. Cuanto más precisos y relevantes sean los resultados, mayor será la utilidad y satisfacción del usuario.

2. **Optimización del Sistema**: Al proporcionar métricas cuantitativas, las medidas de evaluación permiten a los desarrolladores identificar áreas de mejora en el SRI. Esto puede implicar ajustes en algoritmos de búsqueda, ponderación de términos, o mejoras en la indexación de documentos, entre otros.

3. **Comparación de Métodos y Sistemas**: Las medidas de evaluación facilitan la comparación entre diferentes métodos de búsqueda y sistemas de recuperación de información. Esto es fundamental para seleccionar la mejor solución en función de los requisitos específicos del usuario y del contexto de aplicación.

No obstante, crear los conjuntos de datos para aplicar las distintas métricas y evaluar el sistema es un laborioso trabajo que solo puede ser llevado por expertos en el tema.


Para ser consecuentes, continuaremos trabajando con los documentos del corpus 'Cranfield'.

In [ ]:
import ir_datasets
# dataset = ir_datasets.load("cranfield")

La variable **dataset**, instancia de la clase **ir_datasets.datasets.base.Dataset**, tiene 3 funciones con las cuales se estarán trabajando en esta clase. Estas son:

1. **docs_iter()**: Devuelve un objeto iterable de tuplas de dimensión 5, referentes a los documentos. Los campos de cada tupla son:
  - Identificador (str)
  - Título (str)
  - Texto (str)
  - Autor (str)
  - Referencia bibliográfica (str)
  
2. **queries_iter()**: Devuelve un objeto iterable de tuplas de dimensión 2, referentes a las consultas consultas predefinidas. Los campos de cada tupla son:
  - Identificador (str)
  - Text (str)
  
3. **qrels_iter()**: Devuelve un objeto iterable de tuplas de dimensión 5, referentes al nivel de relevancia de los documentos dada una consulta. Los campos de cada tupla son:
  - Identificador de la consulta (str)
  - Identificador del documento (str)
  - Relevancia (int)
    - -1: Sin importancia o valor
    - 1: Referencia de interés mínimo
    - 2: Referencia útil
    - 3: Referencia de alto grado de relevancia
    - 4: Referencia que responde a la consulta
  - Número de iteración (int = 0)


## Usando sklearn 

Scikit-learn, comúnmente conocido como `sklearn`, es una biblioteca de software de código abierto que proporciona herramientas simples y eficientes para el análisis de datos y la modelización estadística. Esta biblioteca es ampliamente utilizada en el campo del aprendizaje automático para la implementación de algoritmos de clasificación, regresión, agrupamiento, y reducción de dimensionalidad. Además, `sklearn` puede utilizarse para medir el desempeño de un SRI a través de métricas de evaluación como precisión, recall, y F1-score, o la matriz de confusión.

In [ ]:
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score

In [ ]:
help(confusion_matrix)

Ejemplo: Asumamos que el corpus cuente con solo 10 documentos y para cierta consulta se tiene los documentos recuperados, siendo estos

In [ ]:
# Etiquetas verdaderas
#   0 -> documento relevante
#   1 -> documento irrelevante
y_true = [0, 1, 1, 0, 1, 0, 1, 0, 0, 1]  

# Predicciones del modelo SRI
#   0 -> documento recuperado
#   1 -> documento no recuperado
y_pred = [0, 1, 1, 0, 0, 0, 1, 1, 0, 0]  # Predicciones del modelo


## Matriz de Confusión

La matriz de confusión es una herramienta de análisis que permite la evaluación del desempeño de un sistema de clasificación. Se estructura en cuatro componentes esenciales:

* Verdaderos Positivos (TP): Número de elementos correctamente identificados como relevantes.
* Falsos Positivos (FP): Número de elementos incorrectamente identificados como relevantes.
* Verdaderos Negativos (TN): Número de elementos correctamente identificados como no relevantes.
* Falsos Negativos (FN): Número de elementos incorrectamente identificados como no relevantes.

Esta matriz proporciona una base para calcular otras métricas de evaluación de desempeño.

In [ ]:
matrix = confusion_matrix(y_true, y_pred)
tn, fp, fn, tp = matrix.ravel()

print(matrix)

## Precisión

La precisión indica qué proporción de los elementos recuperados son relevantes para la consulta realizada y se calcula como:
$$ P = \frac{|TP|}{|TP| + |FP|} $$

In [ ]:
precision = precision_score(y_true, y_pred)
print(f"Precisión: {precision}")

**Nota:** Una precisión de 1 no necesariamente implica que el sistema haya recuperado todos los elementos relevantes disponibles. La precisión se enfoca exclusivamente en la calidad de los resultados recuperados, no en la completitud. Por lo tanto, un sistema podría tener una precisión perfecta pero un recobrado bajo si solo recupera una pequeña fracción de todos los ítems relevantes disponibles.

## Recobrado

El recobrado representa la proporción de elementos relevantes que fueron recuperados con respecto a todos los elementos relevantes existentes en el conjunto de datos, a partir de la consulta. Se calcula como:
$$ \text{R} = \frac{|TP|}{|TP| + |FN|} $$

In [ ]:
recall = recall_score(y_true, y_pred)
print(f"Recobrado: {recall}")

**Nota:** Un recobrado de 1 no indica nada sobre la cantidad de elementos no relevantes que también puedan haber sido recuperados (esto sería evaluado por la precisión). Por lo tanto, es posible tener un recobrado de 1 con una precisión baja si el sistema recupera muchos elementos irrelevantes junto con todos los relevantes. 

## Medida F

La medida F es una métrica que combina precisión y recobrado en un solo valor, lo que proporciona una evaluación equilibrada del sistema. Se calcula como la media armónica de estas medidas:

$$ F = \frac{(1 + \beta^2) \cdot \text{P} \cdot \text{R}}{\beta^2 \cdot \text{P} + \text{R}} $$

donde **β** es un factor el cual indica la medida a prestarle más atención o importancia.

Cuando:
- **β > 1**, el recobrado se considera más importante que la precisión. Por ejemplo, F2 pone más énfasis en el recobrado.
- **β < 1**, la precisión se considera más importante. Por ejemplo, F0.5 pone más énfasis en la precisión.
- **β = 1**, la precisión y el recobrado se consideran igualmente importantes, lo que nos lleva a la medida F1.

### Medida F1

$$ F1 = \frac{2 \cdot \text{P} \cdot \text{R}}{\text{P} + \text{R}}

In [ ]:
f1 = f1_score(y_true, y_pred)
print(f"Medida F1: {f1}")

**Nota:** La razón por la que se utiliza la media armónica en lugar de la media aritmética es que la media armónica penaliza más fuertemente los valores extremos. Esto significa que si uno de los dos valores (precisión o recobrado) es muy bajo, la medida F1 también será baja, lo que refleja una necesidad de equilibrio entre capturar todos los elementos relevantes (recobrado alto) y asegurar que los elementos recuperados sean relevantes (alta precisión).

## R-Precisión

La **R-Precisión** es una métrica específica utilizada para evaluar la efectividad de SRI, enfocándose en los primeros R resultados devueltos por una consulta. Esta medida evalúa la proporción de elementos relevantes encontrados dentro de los primeros R elementos recuperados, donde R es un número predefinido que representa un umbral de interés para el evaluador o el contexto de uso. 

La R-Precisión es particularmente relevante en aplicaciones donde los usuarios tienden a considerar solo los primeros resultados de una búsqueda, como en motores de búsqueda en Internet o en sistemas de recomendación. Al medir la precisión en este subconjunto inicial de resultados, la medida proporciona una visión clara de la calidad y relevancia de los elementos que los usuarios ven primero, lo que es crucial para la satisfacción del usuario y la eficacia general del sistema en entornos prácticos.

In [ ]:
# El primer paso es ordenar de forma descendente según probabilidad las listas y_true y y_pred 
r = 8
r_precision = precision_score(y_true, y_pred[:r] + [0] * (len(y_pred) - r))
print(f"R-Precisión: {r_precision}")

## Proporción de fallo

La proporción de fallo es una métrica que cuantifica la tasa de falsos positivos, es decir, evalúa qué porcentaje de los elementos no relevantes ha sido erróneamente clasificado como relevante por el sistema, en relación con el total de elementos no relevantes. Se expresa mediante la fórmula:
$$ \text{Fallout} = \frac{|FP|}{|FP| + |TN|} $$
Esta medida es importante para evaluar la proporción de ruido generado por el sistema al recuperar información no pertinente.

Esta medida no se encuentra en sklearn puesto que es propia de los SRI.

In [ ]:
fallout = fp / (fp + tn)
print(f"Fallout: {fallout}")

Luego, sklearn provee la mayoría de las medidas necesarias pero tiene un problema: necesita todo el conjunto de datos clasificado. Esto es un problema en cuanto al espacio de memoria cuando conjunto de datos excede la cifra de los millones, el cual es un número común en sistemas que trabajan, cargan y procesan datos constantemente de Internet.

Por tanto, **se requiere que implemente cada métrica pero optimizadas a los subconjuntos de datos relevantes y datos recuperados**. 

Para ello, se brinda 2 funciones que puede utilizar.

In [ ]:
def relevant_documents(query_id : str):
  """
  Returns relevant documents given a query and the query
  
  Args: 
    - query_id (str) : Query identifier.

  Return: 
    list<str>
  
  """
  for (queryt_id, query_text) in dataset.queries_iter():
    if queryt_id == query_id:
      break
    
  return (
    [
      doc_id
      for (queryt_id, doc_id, relevance, iteration) in dataset.qrels_iter()
      if queryt_id == query_id and relevance in [3, 4]
    ], 
    query_text)

import random
def recovered_documents_sri(query):
  """
  Determines the set of documents recovered. The most important one is in position zero and thus the relevance decreases.

  Args:
    - query (str): Query text.

  Return:
    list: List of document identifiers

  """
  document_identifiers = [t[0] for t in dataset.docs_iter()]
  random.shuffle(document_identifiers)
  recovered_documents = document_identifiers[:random.randint(1, len(document_identifiers) - 1)]
  return recovered_documents

Luego, se requiere que implemente cada una de las métricas que a continiación se definen.


In [ ]:
def precision(recovered_documents, relevant_documents):
  """
  Calculate the measure (accuracy)

  Args:
    - recovered_documents (list): Set of documents recovered by the SRI. Each document is defined by its identifier.
    - relevant_documents (list): Set of relevant documents. Each document is defined by its identifier.

  Return:
    double: Value between 0 and 1.

  """
  #! Not Implemented

  return 0

In [ ]:
def recall(recovered_documents, relevant_documents):
  """
  Calculate the measure

  Args:
    - recovered_documents (list): Set of documents recovered by the SRI. Each document is defined by its identifier.
    - relevant_documents (list): Set of relevant documents. Each document is defined by its identifier.
​
  Return:
    double: Value between 0 and 1.

  """
  #! Not Implemented

  return 0

In [ ]:
def f(recovered_documents, relevant_documents):
  """
  Calculate the measure

  Args:
    - recovered_documents (list): Set of documents recovered by the SRI. Each document is defined by its identifier.
    - relevant_documents (list): Set of relevant documents. Each document is defined by its identifier.
​
  Return:
    double: Value between 0 and 1.

  """
  #! Not Implemented

  return 0

In [ ]:
def f1(recovered_documents, relevant_documents):
  """
  Calculate the measure

  Args:
    - recovered_documents (list): Set of documents recovered by the SRI. Each document is defined by its identifier.
    - relevant_documents (list): Set of relevant documents. Each document is defined by its identifier.
​
  Return:
    double: Value between 0 and 1.

  """
  #! Not Implemented

  return 0

In [ ]:
def r_precicion(recovered_documents, relevant_documents, r):
  """
  Calculate the measure

  Args:
    - recovered_documents (list): Set of documents recovered by the SRI. Each document is defined by its identifier.
    - relevant_documents (list): Set of relevant documents. Each document is defined by its identifier.
    - r (int): Ranking position to apply the cut​

  Return:
    double: Value between 0 and 1.

  """
  #! Not Implemented

  return 0

In [ ]:
def fallout(recovered_documents, relevant_documents, r):
  """
  Calculate the measure

  Args:
    - recovered_documents (list): Set of documents recovered by the SRI. Each document is defined by its identifier.
    - relevant_documents (list): Set of relevant documents. Each document is defined by its identifier.
    - r (int): Ranking position to apply the cut​

  Return:
    double: Value between 0 and 1.

  """
  #! Not Implemented

  return 0

Para poder verificar las funciones, tome una consulta y determine por su sistema el conjunto de documentos recuperados. Luego, utilice la función siguiente para el apoyo visual.

In [ ]:
relevant_documents, query_text = relevant_documents('18')
recovered_documents = recovered_documents_sri(query_text)

print(f"""
Identificador de la consulta: {query_id}
Consulta: {query_text}

Métricas:
  Precisión: {precision(recovered_documents, relevant_documents)}
  Recobrado: {recall(recovered_documents, relevant_documents)}
  F: {f(recovered_documents, relevant_documents)}
  F1: {f1(recovered_documents, relevant_documents)}
  R-Precisión: {r_precicion(recovered_documents, relevant_documents, r)}
  Fallout: {fallout(recovered_documents, relevant_documents, r)}

""")